# This file begins pulling senate contributions into a dataframe

In [6]:
import requests,json,pprint
import pandas as pd
import pprint
from api_keys import lda_senate_api_key

headers = {"Authorization": f"Token {lda_senate_api_key}"}

url = "https://lda.senate.gov/api/v1/contributions"
response = requests.get(url,headers=headers).json()
#pprint.pprint(response)
response

{'count': 535116,
 'next': 'https://lda.senate.gov/api/v1/contributions/?page=2',
 'previous': None,
 'results': [{'url': 'https://lda.senate.gov/api/v1/contributions/99f60900-30d1-4799-ae50-ad17954ab91b/',
   'filing_uuid': '99f60900-30d1-4799-ae50-ad17954ab91b',
   'filing_type': 'MM',
   'filing_type_display': 'Mid-Year Report',
   'filing_year': 2008,
   'filing_period': 'mid_year',
   'filing_period_display': 'Mid-Year (Jan 1 - Jun 30)',
   'filing_document_url': 'https://lda.senate.gov/filings/public/contribution/99f60900-30d1-4799-ae50-ad17954ab91b/print/',
   'filing_document_content_type': 'text/html',
   'filer_type': 'lobbyist',
   'filer_type_display': 'Lobbyist',
   'dt_posted': '2008-07-22T15:45:19-04:00',
   'contact_name': None,
   'comments': None,
   'address_1': '1817 Edgehill Drive',
   'address_2': None,
   'city': 'Alexandria',
   'state': 'VA',
   'state_display': 'Virginia',
   'zip': '22307',
   'country': 'US',
   'country_display': 'United States of America',

In [2]:
# This section does a couple things:
# 1. Begins populating new_dict_list with information we want. Later it is converted to a df
# 2. Counts how many results exist and how many contributions exist in each result to make sense 
# of what is being pulled versus what is present. Essentially, to make sure all are accounted for.
# 3. It only loops through the first three pages as a test

new_dict_list = []

# Even though the URL is already defined above, if you don't define it again here,
# it will start at the last page left off as it keeps updating the global variable with the last page.
url = "https://lda.senate.gov/api/v1/contributions/?page=10001"


# Reset cummulativeContributions
cummulativeContributions = 0

# Loop through first three pages
for i in range(5000):
    #url = f"https://lda.senate.gov/api/v1/contributions/?page={i}"
    print(url)
    response = requests.get(url,headers=headers).json()
    resultCount = 0
    totalContributions = 0
    for eachResult in response["results"]:
        contributionCount = 0
        for eachContribution in eachResult["contribution_items"]:
            
            # Store information from response as new entry in dictionary
            new_entry = {"amount":eachContribution["amount"],
                         "contributor_name":eachContribution["contributor_name"],
                         "date":eachContribution["date"],
                         "honoree_name":eachContribution["honoree_name"],
                         "state":eachResult["state"],
                         "country":eachResult["country"]}      
            new_dict_list.append(new_entry)
            contributionCount =contributionCount+1
        #print(f"Contribution count is {contributionCount}")
        resultCount =resultCount+1
        totalContributions = totalContributions + contributionCount
    print(f"Result count is {resultCount}")
    print(f"Total Contributions for this page is {totalContributions}")
    cummulativeContributions = cummulativeContributions + totalContributions
    url = response["next"]
        
    
print(f"Cummulative contributions is {cummulativeContributions}")
df = pd.DataFrame(new_dict_list)

df.shape

https://lda.senate.gov/api/v1/contributions/?page=10001
Result count is 25
Total Contributions for this page is 218
https://lda.senate.gov/api/v1/contributions/?page=10002
Result count is 25
Total Contributions for this page is 302
https://lda.senate.gov/api/v1/contributions/?page=10003
Result count is 25
Total Contributions for this page is 449
https://lda.senate.gov/api/v1/contributions/?page=10004
Result count is 25
Total Contributions for this page is 131
https://lda.senate.gov/api/v1/contributions/?page=10005
Result count is 25
Total Contributions for this page is 220
https://lda.senate.gov/api/v1/contributions/?page=10006
Result count is 25
Total Contributions for this page is 292
https://lda.senate.gov/api/v1/contributions/?page=10007
Result count is 25
Total Contributions for this page is 212
https://lda.senate.gov/api/v1/contributions/?page=10008
Result count is 25
Total Contributions for this page is 130
https://lda.senate.gov/api/v1/contributions/?page=10009
Result count is 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df

In [ ]:
df.to_csv("lda_senate_output/page8001-8100.csv",index=False)